# Chains in LangChain

## Outline

1. LLMChain
- The simplest chain in LangChain.
- It’s just:
  - A PromptTemplate (instructions for the model)
  - An LLM (like ChatOpenAI)
  - An optional OutputParser (to structure the result)
- LLMChain is like a single machine that takes an input and produces an output (e.g., a vending machine → put in a coin, get a snack)

2. Sequential Chains
- let you connect multiple chains together so the output of one becomes the input of the next.

  2.1. SimpleSequentialChain
  - Like a domino line — one output knocks into the next input.
    - Connects chains in a straight line.
    - Only passes a single string output from one chain to the next
    - Very easy to use but limited.
    
  2.2. SequentialChain
  - Like an assembly line in a factory — different stations process different parts, passing results along in an organized way
    - More powerful: handles multiple inputs and outputs.
    - Lets you specify which variable goes to which chain.
    - Useful for more complex workflows

3. Router Chain
- Used when you have multiple specialized chains and you want the system to pick the right one based on the input.
- It uses an LLM as a decision-maker to route the question
- Like a receptionist in an office — listens to your request and sends you to the right department (translation desk, summary desk, etc.)


In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

Note: LLM's do not always produce the same results. When executing the code in your notebook, you may get slightly different answers that those in the video.

In [3]:
# account for deprecation of LLM model
import datetime
# Get the current date
current_date = datetime.datetime.now().date()

# Define the date after which the model should be set to "gpt-3.5-turbo"
target_date = datetime.date(2024, 6, 12)

# Set the model variable based on the current date
if current_date > target_date:
    llm_model = "gpt-3.5-turbo"
else:
    llm_model = "gpt-3.5-turbo-0301"

In [5]:
#!pip install pandas

In [4]:
import pandas as pd
df = pd.read_csv('Data.csv')

- df stands for a Pandas DataFrame (a table-like data structure with rows and columns)
- .head() - DataFrame method that returns the first 5 rows by default
    - Useful when you want a quick look at your dataset without printing the whole table ↓

In [6]:
df.head()

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld\n,I loved this product. But they only seem to l...


## LLMChain

The simplest chain in LangChain.
- It’s just:
  - A PromptTemplate (instructions for the model)
  - An LLM (like ChatOpenAI)
  - An optional OutputParser (to structure the result)
- LLMChain is like a single machine that takes an input and produces an output (e.g., a vending machine → put in a coin, get a snack)

In [7]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [8]:
llm = ChatOpenAI(temperature=0.9, model=llm_model)

- define templates with placeholders
- prompt is now a reusable object ↓

In [9]:
prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

- take your LLM and your prompt template and glue them together into a working LangChain chain
- Separates prompt design from model execution
- Lets you plug this chain into bigger workflows (like sequential chains or agents) ↓

In [10]:
chain = LLMChain(llm=llm, prompt=prompt)

- chain.run(product) - call the LLMChain created earlier
- .run() automatically
    - inserts the input Quenn SIze Sheet Set into the prompt template
    - sends formatted prompt to the llm model
    - returns the model's generated answer as plain text ↓

In [13]:
product = "Luxury Air Mattress"
chain.run(product)

'"Elevate Dream Beds"'

## SimpleSequentialChain

In [12]:
from langchain.chains import SimpleSequentialChain

1. define the model
2. create prompt template
3. build the chain 

This is a pipeline machine: input a product, get back a creative company name ↓

In [14]:
llm = ChatOpenAI(temperature=0.9, model=llm_model)

# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

4. define another prompt template
5. create another chain

This is the second pipeline machine: input a company name, get back description ↓

In [15]:
# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    "Write a 20 words description for the following \
    company:{company_name}"
)
# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

This is the step where the 2 chains connect into one pipeline, so they run automatically in sequence:
- SimpleSequentialChain: A LangChain utility that links multiple chains together in a straight line.
    - The output of one chain becomes the input of the next chain.
    - Works only if each chain passes a single string forward (not multiple variables).
- chains=[chain_one, chain_two]: 
    - Run chain_one first → generates a company name from the product. 
    - Then pass that output into chain_two → generates a 20-word description of the company.
- verbose=True. Turns on debug mode.
    - You’ll see in the console/notebook:
        - The input each chain received.
        - The output each chain produced.
    - Super useful for understanding the flow ↓

In [16]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=True
                                            )

- overall_simple_chain: This is your SimpleSequentialChain object
    - It knows:
        - Step 1 → use chain_one to create a company name.
        - Step 2 → use chain_two to write a description from that name
- .run(product)
    - .run() is a convenience method for passing a single string input into the chain.
    - Here product is your variable
- The final output returned by .run(product) is the company description, not the intermediate name. ↓

In [17]:
overall_simple_chain.run(product)



> Entering new SimpleSequentialChain chain...
Elite Slumber Co.
Elite Slumber Co. offers luxury mattresses and bedding for a restful night's sleep. Experience comfort and quality with us.

> Finished chain.


"Elite Slumber Co. offers luxury mattresses and bedding for a restful night's sleep. Experience comfort and quality with us."

↑↑↑
Why This Is Useful:
- Lets you build multi-step pipelines easily.
- You don’t have to manually take the output of chain_one and feed it into chain_two.
- Great for workflows like:
    - Summarize text → Translate summary → Extract keywords
    - Generate company name → Write tagline → Suggest ad copy
    - Product → Name → Description → Marketing Copy


## SequentialChain

In [18]:
from langchain.chains import SequentialChain

In [19]:
llm = ChatOpenAI(temperature=0.9, model=llm_model)

# prompt template 1: translate to english
first_prompt = ChatPromptTemplate.from_template(
    "Translate the following review to english:"
    "\n\n{Review}"
)
# chain 1: input= Review and output= English_Review
chain_one = LLMChain(llm=llm, prompt=first_prompt, 
                     output_key="English_Review"
                    )


In [20]:
second_prompt = ChatPromptTemplate.from_template(
    "Can you summarize the following review in 1 sentence:"
    "\n\n{English_Review}"
)
# chain 2: input= English_Review and output= summary
chain_two = LLMChain(llm=llm, prompt=second_prompt, 
                     output_key="summary"
                    )


In [21]:
# prompt template 3: translate to english
third_prompt = ChatPromptTemplate.from_template(
    "What language is the following review:\n\n{Review}"
)
# chain 3: input= Review and output= language
chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key="language"
                      )


In [22]:

# prompt template 4: follow up message
fourth_prompt = ChatPromptTemplate.from_template(
    "Write a follow up response to the following "
    "summary in the specified language:"
    "\n\nSummary: {summary}\n\nLanguage: {language}"
)
# chain 4: input= summary, language and output= followup_message
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key="followup_message"
                     )


- SequentialChain: LangChain class that links multiple chains in sequence
    - you can pass/return multiple variables
    - each chain can depend on different combinations of inputs/outputs
- chains = [chain_one, chain_two, chain_three, chain_four]
    - This means the overall pipeline will run four sub-chains in order.
        - chain_one: Translate a review into English → produces "English_Review".
        - chain_two: Summarize the review → produces "summary".
        - chain_three: Analyze sentiment → might produce "sentiment".
        - chain_four: Suggest a follow-up message → produces "followup_message"
- input_variables=["Review"]: tell the chain that the initial input is
- output_variables=["English_Review", "summary", "followup_message"]
    - These are the final outputs you want after all chains run.
    - So when you call overall_chain, you’ll get a dictionary ↓

In [23]:
# overall_chain: input= Review 
# and output= English_Review,summary, followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["Review"],
    output_variables=["English_Review", "summary","followup_message"],
    verbose=True
)

Run your multi-step SequentialChain on a real review from your dataset
- df: is your Pandas DataFrame with a column called "Review".
- df.Review[5]: selects the 6th row (index 5, since Python is 0-based)
- When you pass in review, LangChain will:
    - Take the raw review text as the "Review" input.
    - Run it through each chain in sequence (chain_one → chain_two → chain_three → chain_four).
    - Collect the final outputs in a dictionary ↓


In [24]:
review = df.Review[5]
overall_chain(review)



> Entering new SequentialChain chain...

> Finished chain.


{'Review': "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\nVieux lot ou contrefaçon !?",
 'English_Review': "I find the taste mediocre. The foam doesn't hold, it's weird. I buy the same ones in stores and the taste is much better... Old batch or counterfeit!?",
 'summary': 'The reviewer finds the taste of the product mediocre, with poor foam retention, leading them to suspect that it might be an old batch or counterfeit.',
 'followup_message': "Je suis désolé d'entendre que vous avez trouvé le goût du produit médiocre et que la mousse ne se retient pas bien. Il est possible que vous ayez reçu un lot ancien ou contrefait. Je vous suggère de contacter le fabricant pour clarifier la situation et peut-être demander un remplacement si nécessaire. Merci de partager votre expérience avec nous."}

↑
↑
↑
Why This Matters
- This shows how to apply your chain to real data instead of toy strings.
- Perfect for analyzing many customer reviews in a dataset.
- You can loop this over your whole df.Review column to generate structured outputs for every review.

## Router Chain

- defining physics, maths, history and computer science personas ↓

In [25]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

- setting up a catalog of expert prompt options that can later be used in a RouterChain
- prompt_infos: a list of dictionaries
    - having them in a list makes it easy to loop over, feed into RouterChain, or dynamically build chains ↓

In [26]:
prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    },
    {
        "name": "History", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "computer science", 
        "description": "Good for answering computer science questions", 
        "prompt_template": computerscience_template
    }
]

In [27]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate

In [28]:
llm = ChatOpenAI(temperature=0, model=llm_model)

This code takes your expert prompt templates (prompt_infos) and actually turns them into runnable LLM chains, then prepares a list of destinations for a RouterChain.
1. Create dictionary of chains destination_chains = {}
2. loop over prompt_infos
3. build a ChatPromptTemplate
4. create an LLMCHain for each expert
5. store in dictionary
6. create a human-readable list of destinations
7. join into one string ↓

In [29]:

destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain  
    
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

- setting up a default chain for your Router system. 
- It’s basically a fallback expert to handle cases where the router doesn’t know where to send the question ↓

In [30]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

- this is a router prompt template. 
- It’s what the LLM itself uses to decide which expert chain to route a question to ↓

In [31]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ "DEFAULT" or name of the prompt to use in {destinations}
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: The value of “destination” MUST match one of \
the candidate prompts listed below.\
If “destination” does not fit any of the specified prompts, set it to “DEFAULT.”
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

- how the router LLM is actually built from the template you defined earlier
1. Fill the router template
2. Create a PromptTemplate
3. Build the LLMRouterChain
    Creates an LLMRouterChain that uses your chosen LLM (llm) to:
        - Take the user input.
        - Fill the router prompt (router_prompt).
        - Ask the LLM to decide which expert chain to use.
        - Parse the LLM’s JSON response with RouterOutputParser ↓


In [32]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

- final assembly step where you connect the router with all your expert chains (physics, math, history, CS) and the fallback default chain
- 1. MultiPromptChain:  LangChain class that combines:
    - A router (decides which expert to use).
    - A set of destination chains (the expert personas).
    - A default chain (fallback if no expert fits)
- router_chain=router_chain: The router you built earlier with LLMRouterChain
- destination_chains=destination_chains: The dictionary of expert chains you created
- default_chain=default_chain: A general-purpose fallback chain.
    - If the router decides "destination": "DEFAULT", then this chain runs instead ↓

In [33]:
chain = MultiPromptChain(router_chain=router_chain, 
                         destination_chains=destination_chains, 
                         default_chain=default_chain, verbose=True
                        )

In [34]:
chain.run("What is black body radiation?")



> Entering new MultiPromptChain chain...
physics: {'input': 'What is black body radiation?'}
> Finished chain.


"Black body radiation is the electromagnetic radiation emitted by a perfect absorber and emitter of radiation, known as a black body. A black body absorbs all radiation that falls on it and emits radiation across the entire electromagnetic spectrum. The spectrum of black body radiation is continuous and depends only on the temperature of the black body. This phenomenon is described by Planck's law, which states that the intensity and wavelength distribution of the radiation emitted by a black body is determined solely by its temperature."

In [35]:
chain.run("what is 2 + 2")



> Entering new MultiPromptChain chain...
math: {'input': 'what is 2 + 2'}
> Finished chain.


'The answer to 2 + 2 is 4.'

In [36]:
chain.run("Why does every cell in our body contain DNA?")



> Entering new MultiPromptChain chain...
None: {'input': 'Why does every cell in our body contain DNA?'}
> Finished chain.


'Every cell in our body contains DNA because DNA carries the genetic information that determines the characteristics and functions of an organism. DNA contains the instructions for building and maintaining an organism, including the proteins that are essential for cell function and structure. This genetic information is passed down from parent to offspring and is essential for the growth, development, and functioning of all cells in the body. Having DNA in every cell ensures that each cell has the necessary information to carry out its specific functions and contribute to the overall functioning of the organism.'

my questions:

In [37]:
chain.run("What is Markov Chain?")



> Entering new MultiPromptChain chain...
computer science: {'input': 'What is Markov Chain?'}
> Finished chain.


'A Markov Chain is a mathematical system that undergoes transitions from one state to another, with the probability of transitioning to any particular state depending solely on the current state. In other words, it is a stochastic process that satisfies the Markov property, which states that the future state of the system depends only on the present state and not on the sequence of events that preceded it.\n\nMarkov Chains are used in various fields such as statistics, economics, biology, and computer science to model random processes and make predictions about future states based on the current state. They are particularly useful in analyzing systems with a finite number of states and discrete time steps.\n\nIn computer science, Markov Chains are used in applications such as natural language processing, speech recognition, and image processing. They can be used to generate random sequences of events, simulate complex systems, and analyze the behavior of algorithms.'

In [38]:
chain.run("Explain black holes?")



> Entering new MultiPromptChain chain...
physics: {'input': 'Explain black holes?'}
> Finished chain.


'Black holes are regions in space where gravity is so strong that nothing, not even light, can escape from them. They are formed when a massive star collapses in on itself, creating a singularity at its center. The gravitational pull of a black hole is so intense that it warps space and time around it, creating a point of no return called the event horizon. Anything that crosses the event horizon is pulled into the black hole and crushed into the singularity at its center. Black holes come in different sizes, with supermassive black holes found at the centers of galaxies and stellar black holes formed from the remnants of massive stars.'

In [39]:
chain.run("What is the best statistical method for understanding which marketing channels bring ROI?")



> Entering new MultiPromptChain chain...
computer science: {'input': 'What is the best statistical method for understanding which marketing channels bring ROI?'}
> Finished chain.


'The best statistical method for understanding which marketing channels bring ROI is typically through the use of regression analysis. Regression analysis allows you to analyze the relationship between different marketing channels and the return on investment they bring. By collecting data on the various marketing channels used, the corresponding ROI, and other relevant factors, you can use regression analysis to determine which channels are most effective in driving ROI.\n\nAdditionally, you can also use techniques such as A/B testing, cohort analysis, and attribution modeling to further understand the impact of different marketing channels on ROI. These methods can help you identify the most effective channels for your specific business and make data-driven decisions to optimize your marketing strategy.'

In [40]:
chain.run("When did Croatia enter EU?")



> Entering new MultiPromptChain chain...
History: {'input': 'When did Croatia enter EU?'}
> Finished chain.


'Croatia officially became a member of the European Union on July 1, 2013.'

In [41]:
chain.run("How many legs do spiders have?")



> Entering new MultiPromptChain chain...
None: {'input': 'How many legs do spiders have?'}
> Finished chain.


'Spiders have eight legs.'